In [3]:
from keras.models import Model, Input, save_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, ConvLSTM2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import losses
import h5py

DATA_FILE = 'data.h5'

In [4]:
with h5py.File(DATA_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    steering_training = list(f['steering_training'])
    labels_training = list(f['label_training'])
        
    images_testing = list(f['images_testing'])
    steering_testing = list(f['steering_testing'])
    labels_testing = list(f['label_testing'])

In [7]:
activation = 'relu'

# TODO: remodel network using steering data as additional input

input_shape = (64, 64, 3)
pic_input = Input(shape=input_shape)

img_stack = ConvLSTM2D(64, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = ConvLSTM2D(64, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = ConvLSTM2D(32, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = ConvLSTM2D(32, (3, 3), name='convolution3', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = ConvLSTM2D(16, (3, 3), name='convolution4', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.25)(img_stack)

dense_stack = Dense(128, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(64, activation=activation, name='dense1')(dense_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(32, activation=activation, name='dense2')(dense_stack)
dense_stack = Dense(1, name='output', activation='tanh')(dense_stack)

adam = Adam()
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss='mse')


ValueError: Input 0 is incompatible with layer convolution0: expected ndim=5, found ndim=4

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 16, 16, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
__________

In [ ]:
from generator import DriveDataGenerator

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='./logs')

resize_dims = (input_shape[0], input_shape[1])
train_generator = DriveDataGenerator(images_training, labels_training, resize_dims=resize_dims)
validation_generator = DriveDataGenerator(images_testing, labels_testing, resize_dims=resize_dims)

model.fit_generator(generator=train_generator, validation_data=validation_generator,
                    epochs=500, verbose=1, 
                    callbacks=[model_checkpoint, tensorboard_callback])

Epoch 1/500
244/244 [==============================] - 16s 64ms/step - loss: 0.5343 - val_loss: 0.1650
Epoch 2/500
244/244 [==============================] - 11s 44ms/step - loss: 0.1779 - val_loss: 0.1460
Epoch 3/500
244/244 [==============================] - 12s 47ms/step - loss: 0.1629 - val_loss: 0.1429
Epoch 4/500
244/244 [==============================] - 11s 44ms/step - loss: 0.1503 - val_loss: 0.1365
Epoch 5/500
244/244 [==============================] - 12s 48ms/step - loss: 0.1524 - val_loss: 0.1434
Epoch 6/500
244/244 [==============================] - 12s 48ms/step - loss: 0.1433 - val_loss: 0.1247
Epoch 7/500
244/244 [==============================] - 12s 47ms/step - loss: 0.1403 - val_loss: 0.1316
Epoch 8/500
244/244 [==============================] - 11s 46ms/step - loss: 0.1467 - val_loss: 0.1296
Epoch 9/500
244/244 [==============================] - 10s 40ms/step - loss: 0.1373 - val_loss: 0.1307
Epoch 10/500
244/244 [==============================] - 12s 49ms/step - l

In [6]:
model.save('model.h5')